# Importando las librerias y paquetes para entrenar el modelo

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dropout
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_fscore_support

# Data de configuración para el Modelo

In [3]:
height, width = (28, 28) # 28x28 ya que es la dimensionalidad de los datos de kera
batch_size = 128
num_classes = 10
epochs = 12

# Dataset

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], height, width, 1)
x_test = x_test.reshape(x_test.shape[0], height, width, 1)
input_shape = (height, width, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Convert class vectors to binary class matrices

In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Importando las librerias y paquetes de Tensorflow/Keras

In [6]:
from tensorflow.keras.models import Sequential # Para inicializar la NN 
#(como es una Secuencia de layers, lo hago igual que con ANN; no uso la inici. de Graph)
from tensorflow.keras.layers import Convolution2D # Para hacer el paso de convolución
from tensorflow.keras.layers import AveragePooling2D # Para el Pooling step
from tensorflow.keras.layers import Flatten # Para el flattening
from tensorflow.keras.layers import Dense # Para añadir los fully-connected layers hacia el layer de outputs

# Extracción de variables

## Inicialización de la CNN

In [7]:
model = Sequential()

## Paso 1 - La Convolución
### En Convolución: nº filtros, filas, columnas.
### Se define también la dimensión del kernel (mismo para los 6 filtros que se aplican)
### Se pasa de 32 x 32 x 1 a 28 x 28 x 6

In [8]:
model.add(Convolution2D(filters=6,
 kernel_size=(3, 3),
 activation='relu',
 input_shape=input_shape))
model.layers

## Paso 2 - 1er Avg. Pooling
### El tamaño del kernel del avg. pooling es 2x2
### Se pasa de 28 x 28 x 6 a 14 x 14 x 6

In [9]:
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))
model.layers

## Paso 3 - 2nda Convolución
### En este caso, como es un 2º layer, y su input no es las imagenes, sino el otro layer, no pongo input_shape
### Se pasa de 14 x 14 x 6 a 10 x 10 x 16

In [10]:
model.add(Convolution2D(filters=16,
 kernel_size=(3, 3),
 activation='relu'))
model.layers

## Paso 4 - 2ndo Avg. Pooling
### Se pasa de 10 x 10 x 16 a 5 x 5 x 16

In [11]:
model.add(AveragePooling2D(pool_size=(2, 2),
 strides=2))
model.layers

## Paso 5 - Flattening
### Se pasa de 5 x 5 x 16 a 120 x 1

In [12]:
model.add(Flatten())
model.layers

# Red NN para classificación Fully Connected

## Entrada: n_batch x 120
## HL: 120 x 120
## Salida: n_batch x 120

In [13]:
model.add(Dense(units=120, activation='relu'))

## Entrada: n_batch x 120
## HL: 120 x 84
## Salida: n_batch x 84

In [14]:
model.add(Dense(units=84, activation='relu'))

## Entrada: n_batch x 84
## Salida: Clasificación multiclase para 10 categorías
## n_batch x 10

In [15]:
model.add(Dense(units=10, activation = 'softmax'))

## Compilación

In [16]:
model.compile(loss='categorical_crossentropy',
 optimizer='adam',
 metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 6)         60        
_________________________________________________________________
average_pooling2d (AveragePo (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        880       
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

## Fitting the model

In [17]:
model.fit(x_train, y_train,
 batch_size=batch_size,
 epochs=epochs,
 verbose=1,
 validation_data=(x_test, y_test))

Epoch 1/12
469/469 [==============================] - 18s 35ms/step - loss: 0.3984 - accuracy: 0.8820 - val_loss: 0.1462 - val_accuracy: 0.9556
Epoch 2/12
469/469 [==============================] - 15s 33ms/step - loss: 0.1123 - accuracy: 0.9663 - val_loss: 0.0818 - val_accuracy: 0.9761
Epoch 3/12
469/469 [==============================] - 12s 26ms/step - loss: 0.0830 - accuracy: 0.9750 - val_loss: 0.0721 - val_accuracy: 0.9777
Epoch 4/12
469/469 [==============================] - 15s 32ms/step - loss: 0.0681 - accuracy: 0.9797 - val_loss: 0.0554 - val_accuracy: 0.9822
Epoch 5/12
469/469 [==============================] - 17s 37ms/step - loss: 0.0561 - accuracy: 0.9833 - val_loss: 0.0504 - val_accuracy: 0.9843
Epoch 6/12
469/469 [==============================] - 18s 39ms/step - loss: 0.0488 - accuracy: 0.9844 - val_loss: 0.0479 - val_accuracy: 0.9858
Epoch 7/12
469/469 [==============================] - 13s 27ms/step - loss: 0.0415 - accuracy: 0.9869 - val_loss: 0.0473 - val_accuracy:

# Evaluación del modelo

In [19]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.042724307626485825
Test accuracy: 0.9883999824523926


## Predicciones del test set

In [20]:
y_pred = model.predict(x_test).round()

In [21]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit([1,2,3,4,5,6,7,8,9,10])
y_pred = lb.inverse_transform(y_pred)
y_test = lb.inverse_transform(y_test)

## Ultima Evaluacion

In [23]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: ", cm)
precision, recall, fbeta, support = precision_recall_fscore_support(y_test,
y_pred)
print(precision)

Confusion Matrix:  [[ 978    0    0    0    0    0    0    1    1    0]
 [   3 1131    0    0    0    0    0    0    1    0]
 [   3    0 1018    2    1    0    0    5    3    0]
 [   2    0    1  992    0    1    0    3    4    7]
 [   0    0    2    0  974    0    1    1    0    4]
 [   4    0    0    7    0  876    1    0    4    0]
 [  10    3    0    1    2    1  939    1    1    0]
 [   2    0    2    0    0    0    0 1016    1    7]
 [   5    0    2    1    0    1    1    3  958    3]
 [   1    0    0    1    5    1    0    1    0 1000]]
